# 投票分类器 (Voting Classifier)

## 算法原理

投票分类器是最简单的集成方法，通过组合多个不同类型的分类器进行预测：

1. **硬投票 (Hard Voting)**: 每个分类器投票选择类别，最终选择得票最多的类别
2. **软投票 (Soft Voting)**: 对各分类器的预测概率取平均，选择平均概率最高的类别

## 投票策略对比

| 策略 | 原理 | 要求 | 适用场景 |
|------|------|------|----------|
| 硬投票 | 多数表决 | 无 | 分类器不输出概率时 |
| 软投票 | 概率平均 | 分类器支持 `predict_proba` | 通常效果更好 |

## 适用场景

- 已有多个独立训练的模型
- 希望以最小改动获得性能提升
- 各分类器的错误不相关（互补性强）

In [ ]:
# 导入必要的库
import numpy as np
from sklearn.datasets import make_moons
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

# 设置随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 1. 数据准备

In [ ]:
# 生成月牙形数据集
X, y = make_moons(n_samples=500, noise=0.30, random_state=RANDOM_STATE)

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print(f"训练集大小: {X_train.shape[0]}")
print(f"测试集大小: {X_test.shape[0]}")

## 2. 硬投票分类器

硬投票通过多数表决来确定最终类别。

In [ ]:
# 定义基分类器
log_clf = LogisticRegression(random_state=RANDOM_STATE)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
svc_clf = SVC(random_state=RANDOM_STATE)

# 创建硬投票分类器
voting_hard = VotingClassifier(
    estimators=[
        ('lr', log_clf),
        ('rf', rf_clf),
        ('svc', svc_clf)
    ],
    voting='hard'              # 硬投票
)

# 训练
voting_hard.fit(X_train, y_train)
hard_accuracy = voting_hard.score(X_test, y_test)

print(f"硬投票准确率: {hard_accuracy:.4f}")

## 3. 软投票分类器

软投票基于预测概率进行平均，通常效果更好。

**注意**: SVC 需要设置 `probability=True` 才能输出概率。

In [ ]:
# 创建支持概率输出的分类器
log_clf_soft = LogisticRegression(random_state=RANDOM_STATE)
rf_clf_soft = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
svc_clf_soft = SVC(probability=True, random_state=RANDOM_STATE)  # 启用概率输出

# 创建软投票分类器
voting_soft = VotingClassifier(
    estimators=[
        ('lr', log_clf_soft),
        ('rf', rf_clf_soft),
        ('svc', svc_clf_soft)
    ],
    voting='soft'              # 软投票
)

# 训练
voting_soft.fit(X_train, y_train)
soft_accuracy = voting_soft.score(X_test, y_test)

print(f"软投票准确率: {soft_accuracy:.4f}")

## 4. 与单个分类器对比

In [ ]:
# 训练并评估各个单独的分类器
classifiers = [
    ('Logistic Regression', LogisticRegression(random_state=RANDOM_STATE)),
    ('Random Forest', RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)),
    ('SVC', SVC(probability=True, random_state=RANDOM_STATE)),
    ('Hard Voting', voting_hard),
    ('Soft Voting', voting_soft)
]

results = {}
print("模型性能对比:")
print(f"{'模型':<25} {'准确率':>10}")
print("-" * 37)

for name, clf in classifiers:
    if name not in ['Hard Voting', 'Soft Voting']:
        clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    results[name] = accuracy
    print(f"{name:<25} {accuracy:>10.4f}")

## 5. 交叉验证评估

In [ ]:
# 软投票交叉验证
cv_scores = cross_val_score(voting_soft, X, y, cv=5, scoring='accuracy')

print(f"交叉验证分数: {cv_scores}")
print(f"平均准确率: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## 6. 单元测试验证

In [ ]:
def test_voting_classifier():
    """投票分类器功能测试"""
    
    # 测试1: 模型应该正确训练
    assert hasattr(voting_soft, 'estimators_'), "模型未正确训练"
    
    # 测试2: 应该有3个基分类器
    assert len(voting_soft.estimators_) == 3, "基分类器数量不正确"
    
    # 测试3: 准确率应该在合理范围
    assert soft_accuracy >= 0.8, f"软投票准确率过低: {soft_accuracy}"
    
    # 测试4: 软投票通常不差于硬投票
    assert soft_accuracy >= hard_accuracy * 0.95, "软投票性能异常"
    
    # 测试5: 集成应该优于或接近最佳单模型
    best_single = max(results['Logistic Regression'], 
                      results['Random Forest'], 
                      results['SVC'])
    assert soft_accuracy >= best_single * 0.95, "集成性能异常"
    
    print("所有测试通过!")

test_voting_classifier()

## 总结

### 投票分类器的优势

1. **简单有效**: 实现简单，通常能获得性能提升
2. **模型无关**: 可以组合任意类型的分类器
3. **鲁棒性强**: 单个模型的错误可被其他模型纠正

### 使用建议

1. 选择差异化的基分类器（如线性模型 + 树模型 + 核方法）
2. 优先使用软投票（需要所有分类器支持概率输出）
3. 确保各分类器的性能相近，避免被弱分类器拖累

### 与其他集成方法的对比

- **vs Bagging**: Voting 组合不同类型的模型，Bagging 组合同类型模型
- **vs Stacking**: Voting 简单平均/投票，Stacking 用元学习器学习组合权重